##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Hub에서 전이 학습 이용

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF 허브 모델 보기</a></td>
</table>

[TensorFlow Hub](http://tensorflow.org/hub)는 이 전에 학습된 TensorFlow 모델들의 리포지토리입니다.

이 튜토리얼에서는 다음을 수행하는 방법을 보여줍니다.

1. `tf.keras`와 함께 TensorFlow Hub의 모델을 사용합니다.
2. TensorFlow Hub의 이미지 분류 모델을 사용합니다.
3. 자신의 이미지 클래스에 대한 모델을 미세 조정하기 위해 간단한 전이 학습을 수행합니다.

## !pip install -U tf-hub-nightly<br>import tensorflow_hub as hub<br><br>from tensorflow.keras import layers

In [ ]:
import matplotlib.pylab as plt

import tensorflow as tf

## ImageNet 분류자

우선, 사전 훈련된 분류기 모델을 사용하여 이미지를 얻고 이미지가 무엇인지 예측하겠습니다. 훈련이 필요하지 않습니다!

### 분류자 다운로드하기

`hub.KerasLayer`를 사용하여 TensorFlow Hub에서 [MobileNetV2 모델](https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2)을 로드합니다. TensorFlow Hub의 [호환되는 모든 이미지 분류기 모델](https://tfhub.dev/s?q=tf2&module-type=image-classification)이 여기에서 작동합니다.

In [ ]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

### 단일 이미지에서 실행하기

모델을 시도하기 위한 단일 이미지를 다운로드합니다.

In [ ]:
import numpy as np
import PIL.Image as Image

grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [ ]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

배치 차원을 추가하고 이미지를 모델로 전달합니다.

In [ ]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

그 결과는 이미지에 대한 각 클래스의 확률을 계산하는 로짓의 1001 요소 벡터입니다.

그래서 최상위 클래스 ID는 argmax로 확인할 수 있습니다:

In [ ]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

### 예측 디코딩하기

예측 클래스 ID를 취하고 `ImageNet` 레이블을 가져와서 예측을 디코딩합니다.

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## 간단한 전이 학습

그러나 원본 ImageNet 데이터세트(사전 훈련된 모델이 훈련된 데이터세트)에 포함되지 않은 클래스가 있는 자체 데이터세트를 사용하여 사용자 지정 분류기를 생성하려는 경우에는 어떻게 될까요?

이를 위해 다음을 수행할 수 있습니다.

1. TensorFlow Hub에서 사전 훈련된 모델을 선택합니다. 및
2. 사용자 정의 데이터세트의 클래스를 인식하도록 맨 위(마지막) 레이어를 다시 훈련시킵니다.

### 데이터세트

이 예제를 위해 TensorFlow의 꽃 데이터세트를 사용합니다.

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

image_dataset_from_directory를 사용하여 디스크 이미지를 사용하는 식으로 이 데이터를 모델에 로드하겠습니다.

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

꽃 데이터세트에는 다섯 개의 클래스가 있습니다.

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

이미지 모델에 대한 TensorFlow Hub의 규칙은 `[0, 1]` 범위의 부동 입력을 예상하는 것입니다. 이를 위해 `Rescaling` 레이어를 사용합니다.

참고: 모델 내부에 `Rescaling` 레이어를 포함할 수도 있습니다. 절충점에 대한 논의는 이 [가이드](https://www.tensorflow.org/guide/keras/preprocessing_layers)를 참조하세요.

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

버퍼링된 프리페치를 사용하여 I/O를 차단하지 않고 디스크에서 데이터를 생성할 수 있도록 하겠습니다. 데이터를 로드할 때 다음 두 가지 중요한 메서드를 사용해야 합니다.

이것들은 데이터를 로드할 때 사용해야 하는 가장 중요한 `tf.data` 메서드 중 일부입니다. 관심 있는 독자는 [tf.data API로 성능 향상](https://www.tensorflow.org/guide/data_performance#prefetching) 가이드에서 디스크에 데이터를 캐시하는 방법과 기타 기술에 대해 자세히 알아볼 수 있습니다.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### 이미지 배치에 대한 분류기를 실행해보자

이제 이미지 배치에 대한 분류기를 실행해봅시다.

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = imagenet_labels[tf.math.argmax(result_batch, axis=-1)]
predicted_class_names

얼마나 많은 예측들이 이미지에 맞는지 검토해봅시다:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

이미지 속성을 가진 `LICENSE.txt` 파일을 보세요.

결과가 완벽과는 거리가 멀지만, 모델이 ("daisy"를 제외한) 모든 것을 대비해서 학습된 클래스가 아니라는 것을 고려하면 합리적입니다.

### 헤드리스 모델을 다운로드하세요

텐서플로 허브는 맨 위 분류층이 없어도 모델을 분배 시킬 수 있습니다. 이는 전이 학습을 쉽게 할 수 있게 만들었습니다.

TensorFlow Hub의 모든 [호환 가능한 이미지 특성 벡터 모델](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2)이 여기에서 작동합니다.

In [ ]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

특성 추출기를 만들어봅시다.

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))

이 것은 각각의 이미지마다 길이가 1280인 벡터가 반환됩니다:

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

### 분류 head를 붙이세요.

이제 `tf.keras.Sequential` 모델에 있는 허브 계층을 포장하고, 새로운 분류 계층을 추가하세요.

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

### 모델을 학습시키세요

`Model.compile`을 사용하여 훈련 프로세스를 구성하고 `tf.keras.callbacks.TensorBoard` 콜백을 추가하여 로그를 생성하고 저장합니다.

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

이제 모델을 학습시키기 위해 `.fit`방법을 사용하세요.

이 예제를 짧게 유지하기 위해 단 10개의 epoch 동안 훈련합니다. 나중에 TensorBoard에서 훈련 진행 상황을 시각화하려면 [TensorBoard 콜백](https://www.tensorflow.org/tensorboard/get_started#using_tensorboard_with_keras_modelfit)을 생성하고 로그를 저장합니다.

In [ ]:
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(image_data, epochs=2,
                              steps_per_epoch=steps_per_epoch,
                              callbacks = [batch_stats_callback])

TensorBoard를 시작하여 각 epoch에 따라 측정 항목이 어떻게 변경되는지 확인하고 다른 스칼라 값을 추적합니다.

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/images/tensorboard_transfer_learning_with_hub.png?raw=1"/> -->

### 예측을 확인하세요

이 전의 계획을 다시하기 위해서, 클래스 이름들의 정렬된 리스트를 첫번째로 얻으세요:

In [ ]:
class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

결과를 계획하세요

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## 당신의 모델을 내보내세요

당신은 모델을 학습시켜왔기 때문에, 저장된 모델을 내보내세요:

In [ ]:
import time
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path, save_format='tf')

export_path

이제 우리는 그것을 새롭게 로딩 할 수 있고, 이는 같은 결과를 줄 것입니다:

In [ ]:
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
abs(reloaded_result_batch - result_batch).max()

In [ ]:
abs(reloaded_result_batch - result_batch).max()

In [ ]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(reloaded_predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## 자세히 알아보기

저장된 모델을 사용하여 추론을 위해 로드하거나 [TensorFlow Lite](https://www.tensorflow.org/lite/convert/) 모델(장치 내 머신 러닝용) 또는 [TensorFlow.js](https://www.tensorflow.org/js/tutorials#convert_pretrained_models_to_tensorflowjs) 모델(JavaScript의 머신 러닝용)로 변환할 수 있습니다.

이미지, 텍스트, 오디오 및 비디오 작업에 TensorFlow Hub의 사전 훈련된 모델을 사용하는 방법을 알아보려면 [더 많은 튜토리얼](https://www.tensorflow.org/hub/tutorials)을 살펴보세요.